In [1]:
import sys
sys.path.append('../..')

import pandas as pd

from pandarallel import pandarallel
from helpers.utils import (
    clean_dataset,
    print_max_values_call,
    get_ps1_or_host,
    match_entity,
    is_same_entity,
    count_uniq_dom_call
)

In [2]:
# folder_path = '../../output/data_raw/2025-02-08_noAct_apis.parquet'
# folder_path = '../../output/data_raw/2024-01-24_SF_apis.parquet' 
# folder_path = '../../output/data_raw/2024-01-24_SG_apis.parquet' 
folder_path = '../../output/data_raw/2024-01-24_AMS_apis.parquet' 

df = pd.read_parquet(folder_path)
df.head()

,init_url,final_url,failed_visit,api,loc_href,loc_origin,loc_protocol,loc_host,loc_hostname,loc_port,loc_pathname,loc_hash,doc_url
0,http://www.tantoday.com/forums/members/dclark....,http://www.tantoday.com/forums/members/dclark....,False,http://ajax.googleapis.com/ajax/libs/jquery/1....,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://www.tantoday.com/forums/members/dclark....,http://www.tantoday.com/forums/members/dclark....,False,http://www.tantoday.com/forums/members/dclark....,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN
2,http://www.tantoday.com/forums/members/dclark....,http://www.tantoday.com/forums/members/dclark....,False,http://www.google-analytics.com/ga.js,3.0,NaN,2.0,NaN,14.0,NaN,1.0,1.0,NaN
3,https://hippocampus.org/#1_7,https://hippocampus.org/#1_7,False,https://hippocampus.org/HippoCampus/script/hc....,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://hippocampus.org/#1_7,https://hippocampus.org/#1_7,False,https://www.googletagmanager.com/gtm.js?id=GTM...,NaN,NaN,1.0,NaN,14.0,NaN,NaN,NaN,NaN


In [3]:
# open intersection file

intersection_file = '../../helpers/intersection'

with open(intersection_file, "r") as file:
    intersection = file.read().splitlines()

print(len(intersection))

124202


In [4]:
pandarallel.initialize()

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
df_entity = pd.read_json('../../helpers/domain_map.json')

In [6]:
# cleaning
df = clean_dataset(df, intersection)

In [7]:
df['final_host'] = df.parallel_apply(lambda x: get_ps1_or_host(x['final_url']), axis=1)
df['final_entity'] = df.parallel_apply(lambda x: match_entity(df_entity, x['final_host']), axis=1)
df['api_host'] = df.parallel_apply(lambda x: get_ps1_or_host(x['api']), axis=1)
df['api_entity'] = df.parallel_apply(lambda x: match_entity(df_entity, x['api_host']), axis=1)
df['same_entity'] = df.parallel_apply(lambda x: is_same_entity(x['final_entity'], x['api_entity']), axis=1)

In [8]:
df

,init_url,final_url,failed_visit,api,loc_href,loc_origin,loc_protocol,loc_host,loc_hostname,loc_port,loc_pathname,loc_hash,doc_url,final_host,final_entity,api_host,api_entity,same_entity
3,https://hippocampus.org/#1_7,https://hippocampus.org/#1_7,False,https://hippocampus.org/HippoCampus/script/hc....,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hippocampus.org,hippocampus.org,hippocampus.org,hippocampus.org,hippocampus.org
4,https://hippocampus.org/#1_7,https://hippocampus.org/#1_7,False,https://www.googletagmanager.com/gtm.js?id=GTM...,NaN,NaN,1.0,NaN,14.0,NaN,NaN,NaN,NaN,hippocampus.org,hippocampus.org,googletagmanager.com,Google LLC,different
5,https://hippocampus.org/#1_7,https://hippocampus.org/#1_7,False,https://www.gstatic.com/recaptcha/releases/MHB...,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hippocampus.org,hippocampus.org,gstatic.com,Google LLC,different
6,https://hippocampus.org/#1_7,https://hippocampus.org/#1_7,False,https://consent.cookiebot.com/uc.js?cbid=72bdd...,1.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,hippocampus.org,hippocampus.org,cookiebot.com,Usercentrics GmbH,different
7,https://hippocampus.org/#1_7,https://hippocampus.org/#1_7,False,https://www.googletagmanager.com/gtag/js?id=G-...,9.0,NaN,5.0,NaN,17.0,NaN,2.0,NaN,NaN,hippocampus.org,hippocampus.org,googletagmanager.com,Google LLC,different
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235023,http://www.movilnet.com.ve/sitio/index.jsp?sec...,http://www.movilnet.com.ve/sitio/index.jsp?sec...,False,https://cdnjs.cloudflare.com/ajax/libs/jquery/...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,movilnet.com.ve,movilnet.com.ve,cloudflare.com,"Cloudflare, Inc.",different
2235024,http://www.movilnet.com.ve/sitio/index.jsp?sec...,http://www.movilnet.com.ve/sitio/index.jsp?sec...,False,http://www.movilnet.com.ve/sitio/js/analytics.js,5.0,NaN,5.0,NaN,5.0,NaN,1.0,1.0,NaN,movilnet.com.ve,movilnet.com.ve,movilnet.com.ve,movilnet.com.ve,movilnet.com.ve
2235025,http://www.movilnet.com.ve/sitio/index.jsp?sec...,http://www.movilnet.com.ve/sitio/index.jsp?sec...,False,https://www.googletagmanager.com/gtag/js?id=G-...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,movilnet.com.ve,movilnet.com.ve,googletagmanager.com,Google LLC,different
2235026,http://www.movilnet.com.ve/sitio/index.jsp?sec...,http://www.movilnet.com.ve/sitio/index.jsp?sec...,False,http://www.movilnet.com.ve/sitio/index.jsp?sec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,movilnet.com.ve,movilnet.com.ve,movilnet.com.ve,movilnet.com.ve,movilnet.com.ve


In [9]:
df = df[df['same_entity'] == 'different']

In [10]:
# Show the max values of the columns, third party APIs, and webpages
print_max_values_call(df)

loc_href, 53058.0, https://script.infinity-tracking.com/infinitytrack.js?i=8330, https://www.printglobe.com/quick-ship
loc_origin is empty or all values are NaN.
loc_protocol, 5817.0, https://z.moatads.com/informagamdisplay218733383007/moatad.js, https://www.nojitter.com/customer-experience/wish-2024-better-communication-better-customer-service
loc_host, 8008.0, https://ssp-assets.shelf-ssp.com/9d51b480-b9ec-4333-9ce7-d55235574811/c7915aa9-9477-4788-973c-6725fbcb2a95/custom-branding-script.js, https://www.supersaver.no/support-faq/no-no/30c721c9
loc_hostname, 17025.0, https://security-hub.vaimo.network/static/dbmv2/frontend_module.js, https://www.istore.co.za/education-discount-mac
loc_port, 1528.0, https://epnt.ebay.com/static/epn-smart-tools.js, https://m.roadkillcustoms.com/new-years-day-junkyard-tour-with-chad-nobody-elses-auto/
loc_pathname, 15418.0, https://haon-jpnext.cdn-bebo.com/system-requirement/Web.MobileNew/UA531-01/4d9e20a3b1/main.dcdff51b4ef52bdf.js, https://m.nohu28.com

In [11]:
# Count the number of unique domains
count_uniq_dom_call(df)

{'loc_href': 24485,
 'loc_origin': 0,
 'loc_protocol': 23206,
 'loc_host': 15274,
 'loc_hostname': 23083,
 'loc_port': 3425,
 'loc_pathname': 21606,
 'loc_hash': 19359,
 'doc_url': 8267}